In [7]:
from passlib.context import CryptContext
import sqlite3

class AuthenticationClient:
    """
    This class handles user authentication tasks such as creating users, deleting users, and verifying user credentials.
    """
    def __init__(self, db_file='auth.db'):
        """
        Initialize the AuthenticationClient.

        :param db_file: the path of the SQLite database file
        """
        self.pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
        self.conn = sqlite3.connect(db_file)
        self.conn.execute('''CREATE TABLE IF NOT EXISTS users
                             (username TEXT PRIMARY KEY,
                              password TEXT NOT NULL,
                              role TEXT NOT NULL);''')

    def create_user(self, username, password, role):
        """
        Create a new user.

        :param username: the username
        :param password: the password
        :param role: the role of the user
        :return: True if the user was created, False otherwise
        """
        hashed = self.pwd_context.hash(password)
        try:
            self.conn.execute('INSERT INTO users VALUES (?, ?, ?);', (username, hashed, role))
            self.conn.commit()
            return True
        except sqlite3.IntegrityError:  # Username already exists
            print(f"Username {username} already exists.")
            return False

    def delete_user(self, username):
        """
        Delete a user.

        :param username: the username of the user to delete
        """
        self.conn.execute('DELETE FROM users WHERE username = ?;', (username,))
        self.conn.commit()

    def authenticate(self, username, password):
        """
        Verify the user's credentials.

        :param username: the username
        :param password: the password
        :return: True if the credentials are valid, False otherwise
        """
        cur = self.conn.cursor()
        cur.execute('SELECT password FROM users WHERE username = ?;', (username,))
        row = cur.fetchone()
        if row is None:
            return False  # No such user
        hashed = row[0]
        return self.pwd_context.verify(password, hashed)

    def is_admin(self, username, password):
        """
        Verify if the user is an admin.

        :param username: the username
        :param password: the password
        :return: True if the user is an admin, False otherwise
        """
        cur = self.conn.cursor()
        cur.execute('SELECT password, role FROM users WHERE username = ?;', (username,))
        row = cur.fetchone()

        if row is None:
            return False  # No such user

        stored_hashed_password = row[0]
        role = row[1]
    
        if self.pwd_context.verify(password, stored_hashed_password) and role == 'admin':
            return True

        return False

class Auth:
    """
    This class provides a simple user interface for the AuthenticationClient.
    """
    def __init__(self, username, password, role='user'):
        """
        Initialize the Auth.

        :param username: the username
        :param password: the password
        :param role: the role of the user
        """
        self.auth_client = AuthenticationClient()

        self.username = username
        self.password = password
        self.role = role

    def is_authenticated(self):
        """
        Verify if the user is authenticated.

        :return: True if the user is authenticated, False otherwise
        """
        return self.auth_client.authenticate(self.username, self.password)

    def create_user(self, username, password, role='user'):
        """
        Create a new user if the current user is an admin.
        :param username: the username
        :param password: the password
        :param role: the role of the user
        """
        if self.auth_client.is_admin(self.username, self.password):
            self.auth_client.create_user(username, password, role)
        else:
            print("You are not admin. You cannot create users.")

    def delete_user(self, username):
        """
        Delete a user if the current user is an admin.

        :param username: the username of the user to delete
        """
        if self.auth_client.is_admin(self.username, self.password):
            self.auth_client.delete_user(username)
        else:
            print("You are not admin. You cannot delete users.")

In [8]:
ac = AuthenticationClient()

In [10]:
auth = Auth(username='admin', password='admin')

In [18]:
auth2 = Auth(username='mike', password='mike')

In [17]:
auth.delete_user(username='mike')